# Evaluation TextGrad with TextualVerifier in Optimizer

In [1]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker

## Load Dataset

In [2]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


## Setup Engine

In [3]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/.pyenv/versions/3.11.6/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment Iterations

In [4]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [5]:
def extract_answer_llm(question, final_solution):
    prompt = f"""
    <Question>
    {question}
    </Question>
    
    <Solution>
    {final_solution}
    </Solution>

    <Task>
    Based on last step of solution, which letter (ABCD) selected in question options?
    Response MUST ONLY in 1 letter where LETTER is one of ABCD!
    </Task>
    """
    answer = engine.generate(prompt)
        
    return answer

In [6]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")

        optimizer = TextualGradientDescent([solution])
        
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        # Optimize
        loss_value.backward()
        optimizer.step()

        verification_task_prompts = [
            # Using variables
            """
            1. When using $instruction to $instance, I got $previous context and now in the stage of $calculation.
            2. Evaluate the $calculation value correctly reflects the performance of the instance.
            3. If the $calculation is incorrect or inconsistent, provide the corrected version of the $calculation. 
            4. Do NOT calculate the solution/instance, evaluate $calculation ONLY.
            """
        ]

        verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=verification_task_prompts,
            enable_logging=False
        )

        instance = Variable(f"initial_solution: {row_data['raw_solution']}\nloss_value: {original_loss}",
                            requires_grad=False,
                            role_description="instance")

        optimizer_instruction = Variable("""You will optimize $initial_solution based on $loss_value. Be super concise.""",
                                        requires_grad=False,
                                        role_description="optimizer prompt")

        verified_result = verifier.verify(instance=instance,
                                        instruction=optimizer_instruction,
                                        calculation=solution)

        tracker_data = verifier.get_tracker() # Tracker data  

        final_solution_answer = extract_answer(verified_result.value)

        if final_solution_answer == "Z":
            final_solution_answer = extract_answer_llm(
                question=formatted_question, 
                final_solution=verified_result.value)  

        return {
            "id": row_data["id"],
            "source": row_data["source"],
            "subject": row_data["subject"],
            "question": formatted_question,
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": "",
            "optimized_solution": solution.value,
            "verified_optimized_solution": verified_result.value,
            "final_solution": verified_result.value,
            "final_solution_answer": final_solution_answer, 
            "success": True,
            "error_message": None,
            "processing_time_ms": tracker_data['processing_time_ms'],
            "total_llm_calls": tracker_data['total_llm_calls'],
            "total_input_tokens": tracker_data['total_input_tokens'],
            "total_output_tokens": tracker_data['total_output_tokens'],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "source": "",
            "subject": "",
            "question": "",
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "optimized_solution": "",
            "verified_optimized_solution": "",
            "final_solution": "",
            "final_solution_answer": "", 
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [7]:
def run_evaluation(dataset):
    check_dataset = pd.DataFrame()
    if dataset == "GPQA-Diamond":
        check_dataset = initial_solution[initial_solution["source"] == "GPQA-Diamond"]
    elif dataset == "MMLU-ML":
        check_dataset = initial_solution[initial_solution["source"] == "MMLU-ML"]
    elif dataset == "MMLU-CP":
        check_dataset = initial_solution[initial_solution["source"] == "MMLU-CP"]
    else:
        check_dataset = initial_solution

    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in check_dataset.iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/textgrad-tv-o-1v-using-vars.csv', index=False)
    
    return experiment_df

In [8]:
run_evaluation("All")

Processing:   0%|          | 0/412 [00:00<?, ?it/s]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here's how we can determine the molecular symmetry of the final product:\n\n1. **Nitration of toluene:** Treating toluene (methylbenzene) with nitric acid and sulfuric acid leads to nitration, primarily forming a mixture of ortho- and para-nitrotoluene.  Let's assume the major product is para-nitrotoluene for simplicity.\n\n2. **Oxidation with MnO2 and H2SO4:**  This step oxidizes the methyl group of para-nitrotoluene to a carboxylic acid group, forming para-nitrobenzoic acid.\n\n3. **Reaction with acetone and NaOH:** This is a base-catalyzed aldol condensation.  Acetone reacts with para-nitrobenzoic acid. The carboxylic acid group of para-nitrobenzoic acid reac

Processing:   0%|          | 1/412 [00:06<47:34,  6.95s/it]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The chemist\'s statement "my compounds are on top of each other" implies that they are not successfully separating their target compound from other substances in the reaction mixture.  This suggests a problem with a purification technique, most likely chromatography. In chromatography, compounds are separated based on their differing affinities for a stationary phase and a mobile phase. If compounds have very similar properties, they will elute (come off the column) at similar times, appearing "on top of each other" on the chromatogram or in the collected fractions.\n\nOf the options provided, the most likely reason for this difficulty is similar polarities.  Ch

Processing:   0%|          | 2/412 [00:08<26:37,  3.90s/it]

...

Processing:  92%|█████████▏| 380/412 [03:40<00:17,  1.83it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                        Do not attempt to solve it yourself, do not give a solution, \n                                        only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here\'s how to solve this problem using conservation of momentum and energy:\n\n1. **Conservation of momentum:** Initially, the photon has momentum E/c and the electron is at rest. Finally, the two electrons and the positron, each with mass m, move with equal speed v along the initial direction of the photon. Thus, the final momentum is 3mv.  Therefore, E/c = 3mv.\n\n2. **Conservation of energy:** Initially, the total energy is the photon\'s energy E plus the electron\'s rest energy mc^2. Finally, the total energy is the sum of the total energies of the two electrons and the positron. Since they are moving with speed v, each has energy  mc^2/sqrt(1-v^2/c^2).  Th

Processing: 100%|██████████| 412/412 [04:55<00:00,  1.39it/s]


Completed in 296.1 seconds


,id,source,subject,question,correct_answer,original_solution,original_loss,verified_loss,optimized_solution,verified_optimized_solution,final_solution,final_solution_answer,success,error_message,processing_time_ms,total_llm_calls,total_input_tokens,total_output_tokens,result
0,2,GPQA-Diamond,-,Answer the following multiple choice question....,A,Here's how we can determine the number of carb...,The Corey-Chaykovsky reaction adds a methylene...,,Here's how we can determine the number of carb...,<VERIFIED>Product 1: 9 carbons (from trans-Cin...,<VERIFIED>Product 1: 9 carbons (from trans-Cin...,C\n,True,None,6896.549805,4,1739,119,{'setup': {'verifier_engine': '<textgrad.engin...
1,32,GPQA-Diamond,-,Answer the following multiple choice question....,D,Here's how we can determine the energy of the ...,1. **Incorrect rest mass:** The rest mass of ⁶...,,Here's how we can determine the energy of the ...,<VERIFIED>5601.5 MeV/c² / c² = 5.6015 GeV/c²\n...,<VERIFIED>5601.5 MeV/c² / c² = 5.6015 GeV/c²\n...,D\n,True,None,7412.126465,5,2615,115,{'setup': {'verifier_engine': '<textgrad.engin...
2,36,GPQA-Diamond,-,Answer the following multiple choice question....,C,Here's how we can determine the most likely ob...,No errors.\n,,Here's how we can determine the most likely ob...,<VERIFIED>1. Absorption line energy: 3.9 * 10^...,<VERIFIED>1. Absorption line energy: 3.9 * 10^...,C\n,True,None,15595.175537,9,4503,289,{'setup': {'verifier_engine': '<textgrad.engin...
3,20,GPQA-Diamond,-,Answer the following multiple choice question....,A,The impulse approximation treats the interacti...,Condition 2 is wrong. The probe doesn't inter...,,The impulse approximation treats the interacti...,<VERIFIED>The impulse approximation assumes th...,<VERIFIED>The impulse approximation assumes th...,A\n,True,None,16294.349121,8,5135,459,{'setup': {'verifier_engine': '<textgrad.engin...
4,11,GPQA-Diamond,-,Answer the following multiple choice question....,D,Let's analyze each statement:\n\nA. This state...,No errors identified.\n,,Let's analyze each statement:\n\nA. This state...,<VERIFIED>Evaluate statement A: Statement A ac...,<VERIFIED>Evaluate statement A: Statement A ac...,D\n,True,None,18220.987793,7,3829,323,{'setup': {'verifier_engine': '<textgrad.engin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,391,MMLU-CP,college_physics,Answer the following multiple choice question....,B,We can use Wien's displacement law to solve th...,1. The final answer is misinterpreted. 9.66 µm...,,We can use Wien's displacement law to solve th...,<VERIFIED>(500 x 10^-9 m) * (6000 K) = 3 x 10^...,<VERIFIED>(500 x 10^-9 m) * (6000 K) = 3 x 10^...,C\n,True,None,24476.371826,21,16969,414,{'setup': {'verifier_engine': '<textgrad.engin...
408,409,MMLU-CP,college_physics,Answer the following multiple choice question....,D,The cyclotron frequency *f* is given by the fo...,The units are incorrectly handled. Coulomb ti...,,The cyclotron frequency *f* is given by the fo...,<VERIFIED>m = (2 * 1.6 * 10^-19 C * (π/4) T) /...,<VERIFIED>m = (2 * 1.6 * 10^-19 C * (π/4) T) /...,D\n,True,None,26427.749023,12,8051,631,{'setup': {'verifier_engine': '<textgrad.engin...
409,390,MMLU-CP,college_physics,Answer the following multiple choice question....,D,Here's how we can solve this problem:\n\n1. **...,The primary error lies in step 4 during the de...,,Here's how we can solve this problem:\n\n1. **...,<VERIFIED>Δt = 0\n</VERIFIED>\n<VERIFIED>Δt' =...,<VERIFIED>Δt = 0\n</VERIFIED>\n<VERIFIED>Δt' =...,D\n,True,None,24975.760254,14,15960,520,{'setup': {'verifier_engine': '<textgrad.engin...
410,384,MMLU-CP,college_physics,Answer the following multiple choice question....,A,Here's how we can solve this problem:\n\n1. **...,The relative velocity calculation in step 3 is...,,Here's how we can solve this problem:\n\n1. **...,<VERIFIED>0.6 = 1 * sqrt(1 - v²/c²)\n</VERIFIE...,<VERIFIED>0.6 = 1 * sqrt(1 - v²/c²)\n</VERIFIE...,A\n,True,None,28366.871826,18,19755,778,{'setup'